### Swag Tutorial

In [1]:
from datasets import load_dataset

swag = load_dataset("swag", "regular")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
ending_names = ["ending0", "ending1", "ending2", "ending3"]


def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
tokenized_swag = swag.map(preprocess_function, batched=True)

Found cached dataset swag (/home/laal_intern003/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-4bfcf26e64790f46.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-dc46becd8b80bde6.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-f3a6d5ac5c588cca.arrow


In [9]:
# load dataset
import os, pickle
if os.path.isfile('./data/case_hold.pkl'):
    with open('./data/case_hold_mc.pkl', 'rb') as f:
        data_mapped = pickle.load(f)
else:    
    with open('./data/case_hold.pkl', 'rb') as f:
        data = pickle.load(f)

    def preprocess_function(examples):
        context_sentences = [[context] * 5 for context in examples["context"]]
        ending_sentences = [endings for endings in examples['endings']]
        print(context_sentences)
        print(ending_sentences)
        context_sentences = sum(context_sentences, [])
        ending_sentences = sum(ending_sentences, [])

        tokenized_examples = tokenizer(context_sentences, ending_sentences, truncation=True)
        return {k: [v[i : i + 5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}


    data_mapped = data.map(preprocess_function, batched = True)
    with open('./data/case_hold_mc.pkl', 'wb') as f:
        pickle.dump(data_mapped, f)

In [2]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        print(features)
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")
training_args = TrainingArguments(
    output_dir="my_awesome_swag_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_mapped["train"],
    eval_dataset=data_mapped["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

[{'label': 3, 'input_ids': [[1, 29871, 29946, 29947, 29941, 317, 29889, 29956, 29889, 29906, 29881, 29871, 29906, 29946, 29946, 29892, 29871, 29906, 29946, 29955, 313, 26887, 29889, 29896, 29929, 29955, 29906, 29897, 313, 26740, 11483, 5023, 310, 6534, 9999, 995, 467, 29871, 29906, 29946, 869, 960, 319, 19127, 29915, 29879, 2323, 317, 744, 813, 5320, 2440, 1156, 278, 3974, 813, 310, 29871, 29896, 29892, 29941, 29941, 29900, 7408, 29899, 17536, 1560, 554, 29891, 298, 1446, 304, 4451, 1627, 18375, 414, 508, 10431, 367, 6787, 287, 408, 10127, 292, 263, 9999, 363, 599, 310, 319, 19127, 30010, 29879, 9886, 29871, 29896, 29955, 29892, 29896, 29946, 29900, 8676, 29892, 7408, 29899, 17536, 25158, 29899, 16846, 4063, 298, 1446, 29892, 278, 14260, 8973, 30010, 29879, 24284, 1033, 367, 9120, 491, 5224, 292, 319, 19127, 30010, 29879, 11817, 706, 18658, 9862, 491, 395, 29896, 29892, 29896, 29941, 29929, 29892, 29941, 29945, 29896, 29889, 29906, 29941, 29889, 910, 4377, 338, 15712, 408, 4477, 29901,

../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [647,

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1672, in forward
    outputs = self.bert(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1020, in forward
    encoder_outputs = self.encode

In [6]:
tokenized_swag

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20005
    })
})